In [1]:
import requests
import cv2
import numpy as np
import imutils
import pytesseract
import xmltodict
import json

In [2]:
pytesseract.pytesseract.tesseract_cmd= r'C:\ProgramData\anaconda\envs\mlops\Scripts\pytesseract.exe'  # Teseract Location
cascade= cv2.CascadeClassifier('haarcascade_russian_plate_number.xml')    # Cascade Model
url = "http://192.168.0.102:8080/shot.jpg"
minArea= 500

In [23]:
def capture(img):
    while True:
#         img_resp = requests.get(url)
#         img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8)
#         img = cv2.imdecode(img_arr, -1)
        print(type(img))
        img = np.array(img)
        print(type(img))

        #img = imutils.resize(image=img, width=1000, height=1800, inter=3)

        imgGray= cv2.cvtColor(np.float32(img), cv2.COLOR_BGR2GRAY)
        numberplate= cascade.detectMultiScale(imgGray, 1.1,4)

        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        for (x,y,w,h) in numberplate:
            print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            area= w*h
            if area > minArea:
                cv2.rectangle(img, (x,y), (x+w, y+h), [0,255,0], 2)
                cv2.putText(img,"NUMBER_PLATE", (x, (y-5)), cv2.FONT_HERSHEY_COMPLEX, 1, [0,255,0], 2)
            imgCrop= img[y: y+h, x: x+w]

        cv2.imshow("Number_Plate", img)
        if cv2.waitKey(1) == 13:
            break

        hImg, wImg,_ = imgCrop.shape
        imgRGB= cv2.cvtColor(imgCrop, cv2.COLOR_BGR2RGB)
        cv2.imwrite('../imgRGB.jpg', imgCrop)
    cv2.destroyAllWindows()

def get_vehicle_info(plate_number):
    r = requests.get("http://www.regcheck.org.uk/api/reg.asmx/CheckIndia?RegistrationNumber={0}&username=abrakadabra".format(str(plate_number)))
    data = xmltodict.parse(r.content)
    jdata = json.dumps(data)
    data_final = json.loads(jdata)
    data_print = json.loads(data_final['Vehicle']['vehicleJson'])
    print(data_print)
    return data_print


def text(image):
    def listToString(s):         
        str1 = ""          
        for ele in s: 
            str1 += ele         
        return str1 
    
    print(type(image))

    img = cv2.imread(image)
    hImg, wImg,_ = img.shape
    #img= cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img= cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #(thresh, img) = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

    boxes= pytesseract.image_to_boxes(img)
    #print(boxes)
    plate= []
    for k in boxes.splitlines():
        plate.append(k[0])

    final= listToString(plate)
    if len(final) > 10:
        final = final[0:10]
    #print(final_plate)
        print(final)
    for b in boxes.splitlines():
        b= b.split(' ')
        #print(b)
        x,y,w,h= int(b[1]), int(b[2]), int(b[3]), int(b[4])
        cv2.rectangle(img, (x,y), (w,h),[0,255,0],1)
        cv2.putText(img, b[0], (x,hImg- y+25), cv2.FONT_HERSHEY_COMPLEX, 1, [0,255,0], 2)
    #print(pytesseract.image_to_boxes(img))
    cv2.imshow('RESULT', img)
    cv2.waitKey(0) 
    get_vehicle_info(final)


In [24]:
capture("carimages/car1.jpg")

<class 'str'>
<class 'numpy.ndarray'>


ValueError: could not convert string to float: 'carimages/car1.jpg'

In [ ]:
text('./imgRGB.jpg')